In [1]:
import sys
sys.path.append('../../')

In [2]:
from run_bart_sum import BartSystem
from pathlib import Path
import json
from transformers import BartTokenizer
import spacy
import tqdm

In [3]:
device_ = 0
if device_ > -1:
    device = f'cuda:{device_}'
else:
    device = 'cpu'
model = BartSystem.load_from_checkpoint('../../.models/bart2/epoch=1_v0.ckpt', map_location='cuda:0')
model.model.to('cuda:0')

2020-04-06 12:23:14,385 - INFO - transformers.configuration_utils - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/config.json from cache at /home/dhruveshpate/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d4109359751cb4e739.a42ba2fc4aa449981f74aff39e9b70a82a10ed5f615e41f7fb847905641ecc57
2020-04-06 12:23:14,388 - INFO - transformers.configuration_utils - Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": null,
  "attention_dropout": 0.0,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): SelfAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=Tr

In [4]:
tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')

2020-04-06 12:23:25,729 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json from cache at /home/dhruveshpate/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2020-04-06 12:23:25,731 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt from cache at /home/dhruveshpate/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


In [5]:
spacy_nlp =spacy.load("en_core_web_sm")

In [6]:
# read input data
set_ = 'dev'
input_file = Path(f'../../.data/RACE/set3/{set_}.json')

In [7]:
with open(input_file) as f:
    samples = json.load(f)

In [8]:
def process_batch(batch, model, post_processor):
    inp = [ex['question'] + ' ' + ex['option'] for ex in batch]
    inp_tensor = tokenizer.batch_encode_plus(inp, pad_to_max_length=True, max_length=40, return_tensors='pt')
    out_tensor = model.model.generate(inp_tensor["input_ids"].cuda(),
            attention_mask=inp_tensor["attention_mask"].cuda(),
            num_beams=1,
            max_length=40,
            length_penalty=1000.0, repetition_penalty=2.5,
            early_stopping=True,).cpu()
    preds = [
            tokenizer.decode(
                g, skip_special_tokens=True, clean_up_tokenization_spaces=False)

            for g in out_tensor
        ]
    # post process
    for pred,ex in zip(preds, batch):
        ex['neural_hypothesis'] = (list(post_processor(pred).sents)[0]).text.strip()

In [9]:
batch_size = 128
def batches(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [10]:
for batch in tqdm.tqdm(batches(samples, batch_size), total=len(samples)//batch_size + 1):
    process_batch(batch, model, spacy_nlp)

 99%|█████████▊| 78/79 [16:08<00:12, 12.41s/it]


In [11]:
# write output
output_file = input_file.absolute().parent /(f'{set_}' + '_neural.json')
with open(output_file, 'w') as f:
    json.dump(samples, f)

In [17]:
sublist = samples[10:20]
print('---Question+option')
for s in sublist:
    print(f"{s['question'] +' '+s['option']}")
print('---Rule based---')
for s in sublist:
    print(f"{s['hypothesis']}")
print('---Neural (BART) based---')
for s in sublist:
    print(f"{s['neural_hypothesis']}")

---Question+option
What's the best title for this passage? A busy family
Why are ECAs important now? They can reduce students' learning time.
What can we learn from the text? Red and his friends enjoyed being recorded by the cameras.
What can we infer from the passage? Kmart claimed to ensure the animals' safety but didn't do it very well.
Which of the following is Not true? Many lottery winners buy themselves things with the money that they won.
Which of the following can describe Mini's feeling? Sad.
Which  of the following is right? Xiao Ming 's mother isn't at home.
Why did people throw Mary into the river? To prove that she was guilty
Which of the following is the best title  for this passage? An excellent mother
What topic do the three pieces of information belong to? Society
---Rule based---
The best title for this passage's a busy family.
ECAs are important now because they can reduce students' learning time.
We can learn Red and his friends enjoyed being recorded by the camera